In [1]:
import pandas as pd

base = pd.read_csv('dataset.csv')

In [2]:
import numpy as np

previsores = base.iloc[:1400000,0:20].values
previsores = np.reshape(previsores, (2000, 700, 20))
print(previsores.shape)

classesQ8 = base.iloc[:1400000, 20:28].values
classesQ8 = np.reshape(classesQ8, (2000, 700, 8))
print(classesQ8.shape)

classesQ3 = base.iloc[:1400000, 28:31].values
classesQ3 = np.reshape(classesQ3, (2000, 700, 3))
print(classesQ3.shape)

(2000, 700, 20)
(2000, 700, 8)
(2000, 700, 3)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNGRU, Bidirectional

In [5]:
def criarRede(saida):
    model = Sequential()
  
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True), input_shape = (700, 20)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))
  
    model.add(Dense(saida, activation = 'softmax'))
  
    model.compile(optimizer = 'adam', metrics = ['acc'], loss='categorical_crossentropy')
  
    return model

In [7]:
from sklearn.metrics import balanced_accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

def train_and_evaluate_model(model, x_train, y_train, x_test, y_test, saida):
    model.fit(x_train, y_train, epochs = 100, verbose = 1)
  
    predicted = model.predict(x_test)
    y_teste = []
    predict = []
  
    predicted = np.reshape(predicted, (predicted.shape[0] * predicted.shape[1], saida))
    y_test = np.reshape(y_test, (y_test.shape[0] * y_test.shape[1], saida))
    x_test = np.reshape(x_test, (x_test.shape[0] * x_test.shape[1], 20))

    for i in range(len(x_test)):
        cont = 0
        for j in range(len(x_test[i])):
            cont += x_test[i][j]
        if cont != 0:
            y_teste.append(y_test[i])
            predict.append(predicted[i])
    
    y_teste = np.asarray(y_teste)
    predict = np.asarray(predict)

    predicted = predict
    y_test = y_teste
    
    predicted = np.argmax(predicted, axis=1)
    y_test = np.argmax(y_test, axis=1)

    accu.append(balanced_accuracy_score(y_test, predicted)) 
    precisions.append(precision_score(y_test, predicted, average = 'weighted'))
    recalls.append(recall_score(y_test, predicted, average = 'weighted'))
    f1.append(f1_score(y_test, predicted, average = 'weighted'))
    
    print(classification_report(y_test, predicted))
    print('Acurácia')
    print(balanced_accuracy_score(y_test, predicted))
    print('Precisao')
    print(precision_score(y_test, predicted, average = 'weighted'))
    print('Recall')
    print(recall_score(y_test, predicted, average = 'weighted'))
    print('F1')
    print(f1_score(y_test, predicted, average = 'weighted'))
    print(confusion_matrix(y_test, predicted))

In [8]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle = True)

accu = []
precisions = []
recalls = []
f1 = []

for train_index, test_index in kf.split(previsores):
    model = None
    model = criarRede(8)
    print("TRAIN:", train_index, "TEST:", test_index)
    train_and_evaluate_model(model, previsores[train_index], classesQ8[train_index],
                           previsores[test_index], classesQ8[test_index], 8)

model.summary()

print('Acurácias total')
print(accu)
accu = np.array(accu)
print(accu.mean())
print('Precision total')
print(precisions)
precisions = np.array(precisions)
print(precisions.mean())
print('Recalls total')
print(recalls)
recalls = np.array(recalls)
print(recalls.mean())
print('F1 total')
print(f1)
f1 = np.array(f1)
print(f1.mean())

accu = []
precisions = []
recalls = []
f1 = []

for train_index, test_index in kf.split(previsores):
    model = None
    model = criarRede(3)
    print("TRAIN:", train_index, "TEST:", test_index)
    train_and_evaluate_model(model, previsores[train_index], classesQ3[train_index],
                           previsores[test_index], classesQ3[test_index], 3)
    
model.summary()

print('Acurácias total')
print(accu)
accu = np.array(accu)
print(accu.mean())
print('Precision total')
print(precisions)
precisions = np.array(precisions)
print(precisions.mean())
print('Recalls total')
print(recalls)
recalls = np.array(recalls)
print(recalls.mean())
print('F1 total')
print(f1)
f1 = np.array(f1)
print(f1.mean())

W0822 16:50:10.034723  2648 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0822 16:50:10.043672  2648 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0822 16:50:10.045668  2648 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py

TRAIN: [   1    3    4 ... 1996 1997 1999] TEST: [   0    2    9   16   18   22   23   28   31   34   35   47   49   51
   54   64   66   73   75   77   78   82   84   86   87   90   92   93
   96  101  106  109  113  118  124  125  126  129  140  144  146  147
  161  163  164  176  177  178  181  184  187  189  201  202  208  218
  219  221  238  261  263  287  290  291  292  294  299  301  306  313
  314  325  329  337  365  366  374  385  388  389  390  391  398  399
  411  412  418  422  423  427  432  436  440  450  451  468  479  480
  485  487  493  494  498  507  511  515  516  518  523  526  534  535
  545  549  550  551  556  561  562  565  567  568  575  580  584  587
  592  597  602  603  610  619  628  633  637  639  647  649  661  665
  672  675  681  682  690  702  706  719  730  732  735  742  748  761
  768  778  780  786  787  792  795  798  804  817  826  827  831  833
  841  847  862  863  874  877  905  909  912  913  914  916  925  927
  934  947  950  952  956  9

W0822 16:50:11.548677  2648 deprecation.py:323] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
1600/1600 [==============================] - 21s 13ms/sample - loss: 0.5804 - acc: 0.1602
Epoch 2/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.5286 - acc: 0.1723
Epoch 3/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.5192 - acc: 0.1767
Epoch 4/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.5132 - acc: 0.1791
Epoch 5/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.5084 - acc: 0.1815
Epoch 6/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.5064 - acc: 0.1822
Epoch 7/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.5028 - acc: 0.1841
Epoch 8/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.4976 - acc: 0.1861
Epoch 9/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.4938 - acc: 0.1880
Epoch 10/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.4894 -

1600/1600 [==============================] - 15s 9ms/sample - loss: 0.2988 - acc: 0.2559
Epoch 82/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.2992 - acc: 0.2556
Epoch 83/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.2971 - acc: 0.2565
Epoch 84/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.2956 - acc: 0.2571
Epoch 85/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.2943 - acc: 0.2576
Epoch 86/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.2932 - acc: 0.2580
Epoch 87/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.2923 - acc: 0.2582
Epoch 88/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.2916 - acc: 0.2587
Epoch 89/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.2907 - acc: 0.2586
Epoch 90/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.2913 - acc: 

...

1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1304 - acc: 0.9455
Epoch 96/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1300 - acc: 0.9460
Epoch 97/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1299 - acc: 0.9458
Epoch 98/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1285 - acc: 0.9464
Epoch 99/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1276 - acc: 0.9470
Epoch 100/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1283 - acc: 0.9465
              precision    recall  f1-score   support

           0       0.71      0.74      0.72     43617
           1       0.70      0.63      0.66     23976
           2       0.77      0.78      0.77     39623

    accuracy                           0.73    107216
   macro avg       0.72      0.72      0.72    107216
weighted avg       0.73      0.73      0.73    107216

Acurácia
0.71501023

1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1854 - acc: 0.9211
Epoch 50/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1847 - acc: 0.9216
Epoch 51/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1817 - acc: 0.9229
Epoch 52/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1811 - acc: 0.9229
Epoch 53/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1780 - acc: 0.9243
Epoch 54/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1766 - acc: 0.9252
Epoch 55/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1763 - acc: 0.9250
Epoch 56/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1751 - acc: 0.9257
Epoch 57/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1738 - acc: 0.9264
Epoch 58/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1713 - acc: 

1600/1600 [==============================] - 15s 9ms/sample - loss: 0.3130 - acc: 0.8585
Epoch 4/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.3079 - acc: 0.8609
Epoch 5/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.3060 - acc: 0.8611
Epoch 6/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.3027 - acc: 0.8632
Epoch 7/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.3036 - acc: 0.8628
Epoch 8/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.2969 - acc: 0.8658
Epoch 9/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.2957 - acc: 0.8667
Epoch 10/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.2937 - acc: 0.8676
Epoch 11/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.2929 - acc: 0.8684
Epoch 12/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.2921 - acc: 0.8691

1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1472 - acc: 0.9383
Epoch 84/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1455 - acc: 0.9389
Epoch 85/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1445 - acc: 0.9396
Epoch 86/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1433 - acc: 0.9401
Epoch 87/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1427 - acc: 0.9403
Epoch 88/100
1600/1600 [==============================] - 15s 10ms/sample - loss: 0.1413 - acc: 0.9406
Epoch 89/100
1600/1600 [==============================] - 15s 10ms/sample - loss: 0.1410 - acc: 0.9409
Epoch 90/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1407 - acc: 0.9412
Epoch 91/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.1393 - acc: 0.9416
Epoch 92/100
1600/1600 [==============================] - 15s 10ms/sample - loss: 0.1390 - ac

1600/1600 [==============================] - 15s 10ms/sample - loss: 0.2175 - acc: 0.9056
Epoch 38/100
1600/1600 [==============================] - 15s 10ms/sample - loss: 0.2158 - acc: 0.9068
Epoch 39/100
1600/1600 [==============================] - 15s 10ms/sample - loss: 0.2115 - acc: 0.9088
Epoch 40/100
1600/1600 [==============================] - 15s 10ms/sample - loss: 0.2091 - acc: 0.9098
Epoch 41/100
1600/1600 [==============================] - 15s 10ms/sample - loss: 0.2074 - acc: 0.9108
Epoch 42/100
1600/1600 [==============================] - 15s 10ms/sample - loss: 0.2051 - acc: 0.9112
Epoch 43/100
1600/1600 [==============================] - 15s 10ms/sample - loss: 0.2030 - acc: 0.9123
Epoch 44/100
1600/1600 [==============================] - 15s 10ms/sample - loss: 0.1993 - acc: 0.9142
Epoch 45/100
1600/1600 [==============================] - 15s 10ms/sample - loss: 0.1974 - acc: 0.9150
Epoch 46/100
1600/1600 [==============================] - 15s 9ms/sample - loss: 0.194